# L0 data clean

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import imp
import datetime
import pytz
%matplotlib notebook

In [2]:
#import self-written libraries
import imp
import LVL1
from settings import Glacier, Station, base_path

In [3]:
#Are there newly downloaded data?
new_data=True

In [4]:
if new_data:
    add_new_logger_data_script_pth='"' + base_path + 'Code/Weather/LVL_0/ConcatenateRawWxData.py'+ '"'
    %run $add_new_logger_data_script_pth

 concatinating Q:/Project Data/GlacierData/Benchmark_Program/Data/Gulkana\2013\Wx\gulkana1725_2013_0827.txt
 concatinating Q:/Project Data/GlacierData/Benchmark_Program/Data/Gulkana\2014\Wx\gulkana1725_2014_0826.txt
 concatinating Q:/Project Data/GlacierData/Benchmark_Program/Data/Gulkana\2015\Wx\gulkana1725_2015_0415.txt
 concatinating Q:/Project Data/GlacierData/Benchmark_Program/Data/Gulkana\2015\Wx\gulkana1725_2015_0924.txt
 concatinating Q:/Project Data/GlacierData/Benchmark_Program/Data/Gulkana\2016\Wx\gulkana1725_2016_0828.txt
 concatinating Q:/Project Data/GlacierData/Benchmark_Program/Data/Gulkana\2017\Wx\gulkana1725_20170421.txt
 concatinating Q:/Project Data/GlacierData/Benchmark_Program/Data/Gulkana\2017\Wx\gulkana1725_20170822.txt
 concatinating Q:/Project Data/GlacierData/Benchmark_Program/Data/Gulkana\2018\Wx\gulkana1725_20180423.txt
stripping whitespace from Date
stripping whitespace from Time
stripping whitespace from Instrument
stripping whitespace from Value
strippin

In [5]:
if Glacier + Station =='Wolverine370':
    print ('STOP: not needed here; start at L1. All data from NWIS')

In [6]:
if (Glacier + Station== 'Wolverine1420') | (Glacier + Station== 'Gulkana1920'): #If a JWS station, which logs @ 5 minutes
    if new_data:
        convert5minto15min_script_pth='"' + base_path + 'Code/Weather/LVL_0/ConvertFrom5minTo15minData_JWS.py'+ '"'

        %run $convert5minto15min_script_pth

In [7]:
#Read in data
folder=base_path+ r"Data/" + Glacier+ "/AllYears/Wx/Raw" #folder cont
file_label='_15min_' #used for importting and output name
yr='all'
file=Glacier.lower()+Station+file_label+yr+".csv" #filename; change if yours different
pth=os.path.join(folder, file) #path to data file

In [8]:
#Set constants
date_format='%m/%d/%Y %H:%M:%S' #input date format (change if yours is different)
out_date_format='%Y/%m/%d %H:%M'
timezone='America/Anchorage' #choose from pytz.all_timezones

In [9]:
dat=pd.read_csv(pth)
print("read file: " + file)

read file: gulkana1725_15min_all.csv


In [10]:
dat.tail(8)

,Date,Time,Instrument,Value,Unit,Flag,DateTime
975849,04/23/2018,18:30:00,RD,225.1700,W/m2,G,04/23/2018 18:30:00
975850,04/23/2018,18:30:00,DS,1.9340,meters,G,04/23/2018 18:30:00
975851,04/23/2018,18:30:00,MS,0.7000,MPS,G,04/23/2018 18:30:00
975852,04/23/2018,18:30:00,MD,182.0000,DEG,G,04/23/2018 18:30:00
975853,04/23/2018,18:30:00,T,-1.1000,C,G,04/23/2018 18:30:00
975854,04/23/2018,18:30:00,VB,14.0000,V,G,04/23/2018 18:30:00
975855,04/23/2018,18:30:00,TA,-3.9000,C,G,04/23/2018 18:30:00
975856,04/23/2018,18:30:00,PC,0.8031,m,G,04/23/2018 18:30:00


## Convert Logger time to UTC and Local Time

In [11]:
#Logger records on UTC -8; convert to local time
#Create date-time column
#dat['DateTime']=dat.Date+ " " + dat.Time #this is needed for non-cumulative processing
dat.loc[:,'DateTime']=pd.to_datetime(dat['DateTime'], format=date_format) #set to date-time from string

#Correct for time slips
dat['DateTime']=dat['DateTime'].dt.round('15min') #round time to the nearest 15 minute value

#Convert to UTC time, if logger is logging in UTC-8 (true at some stations)
if (Glacier+Station=='Wolverine990') | (Glacier+Station=='Gulkana1480') | (Glacier+Station=='Gulkana1920') | (Glacier+Station=='Wolverine1420'):
    dat['UTC_col']=dat.DateTime + datetime.timedelta(hours=8)
if (Glacier+Station=='Gulkana1725'):
    dat['UTC_col']=dat.DateTime #logger is already in UTC time
    
#Set timezone of known UTC column
dat['UTC_col'].timezone='UTC'
#Create column for local time
local_timezone=pytz.timezone(timezone) #create local timezone object (e.g. AK time)

dat=dat.set_index('UTC_col')
#Create column for local time
local_timezone=pytz.timezone(timezone) #create local timezone object (e.g. AK time)

#Create local time
dat['local_time'] = dat.index.tz_localize('UTC').tz_convert(local_timezone)

dat['index_local_time']=dat.local_time.copy()

#Set index to local time
dat=dat.set_index('index_local_time')

In [12]:
#Drop duplicates that have same instrument name and date/time stamp (from multiple downloads of same portion of data)
dat=dat.drop_duplicates(subset=['DateTime', 'Instrument'], keep='first') #gets rid of overlaps (i.e. multiple downloads of same bit of data)

In [13]:
dat.head(1)

,Date,Time,Instrument,Value,Unit,Flag,DateTime,local_time
index_local_time,,,,,,,,
2013-03-19 16:45:00-08:00,03/20/2013,00:45:00,MS,7.6,MPS,G,2013-03-20 00:45:00,2013-03-19 16:45:00-08:00


In [14]:
#Instrument Label Change - if same label has been used for different quantities thru time: separate into 2 distinct labels.
if Glacier + Station=="Wolverine990":
    dat.loc[(dat.Instrument=='WD') & (dat.index<'2015-09-06 17:15'), 'Instrument']='WindDir' #before this date, "WD" was simply wind direction; after, is vector-averaged
    dat.loc[(dat.Instrument=='WD') & (dat.index>='2015-09-06 17:15'), 'Instrument']='WDSA' 
    
if Glacier + Station=="Gulkana1480":
    dat.loc[(dat.Instrument=='WD') & (dat.index<'2015-09-28 12:00'), 'Instrument']='WindDir' #before this date, "WD" was simply wind direction; after, is vector-averaged
    dat.loc[(dat.Instrument=='WD') & (dat.index>='2015-09-28 12:00'), 'Instrument']='WDSA' 
    
#Gulkana 1725 has had consistent variable naming throughout

## Consolidate Sensor Naming
#### Has varied thru time

In [15]:
#If site does NOT have a given sensor, leave label, create empty list

if Glacier + Station =='Wolverine990':  
    #Name recorded on the logger changed thru time; indicate which are grouped (i.e. logging same variable) here
    wind_gust_labels=['WindGusts', 'WG', 'WSG']
    wind_speed_labels=['AvgWind', 'WS'] #this is NOT vector-averaged wind speed
    vec_avg_wind_dir=['WDSA']
    wind_dir_labels=['WindDir']
    temp_unasp1_labels=['AirTemp1', 'T1', 'T']
    temp_unasp2_labels=['AirTemp2', 'T2']
    temp_asp1_labels=['AirTempAsp', 'AspT', 'AT']
    temp_asp2_labels=['ASPTRH', 'AT2']
    precip_stage_labels=['PR']
    precip_weighing_labels=['TPG', 'PC']
    rel_hum_labels=['RH']
    barometer_labels=['B']
    logger_bat_label=['BV']
    logger_temp_label=['Tinternal']
    bad_labels=['PrecipStage', 'AirTemp3', 'Air Temp w/FanN', 'AirTemp w/Fan', 'VectorAvgWindDir', 'PS', 'MWD', 'WDA'] #labels containing only bad data; introduced during maintenence etc.
    radiation_up_labels=[]
    radiation_down_labels=[]
    snow_depth_labels=[] 
    
if Glacier + Station =='Gulkana1480':  
    #Name recorded on the logger changed thru time; indicate which are grouped (i.e. logging same variable) here
    wind_gust_labels=['WSG']
    wind_speed_labels=['WindSpeed', 'WS']
    vec_avg_wind_dir=['WDSA']
    wind_dir_labels=['WindDir'] #this is NOT-vector average wind direction
    temp_unasp1_labels=['AirTemp1', 'T1', 'T']
    temp_unasp2_labels=['AirTemp2', 'T2']
    temp_asp1_labels=['AspT', 'AT']
    temp_asp2_labels=['AT2']
    precip_stage_labels=['PR'] #precip doesn't begin until 2010-07; before this, is recording, but logger only has 0 precip
    precip_weighing_labels=['TPG', 'PC']
    rel_hum_labels=['RH']
    barometer_labels=[] #no barometer at this station
    logger_bat_label=['BV']
    logger_temp_label=['Tinternal']
    bad_labels=['PrecipStage', 'AirTemp3', 'Stage'] #labels containing only bad data; introduced during maintenence etc.
    radiation_up_labels=[]
    radiation_down_labels=[]
    snow_depth_labels=[]    
    
if Glacier + Station=='Gulkana1725':
    #Only a single name thru time; no changes @ 1725
    wind_gust_labels=[]#no wind gust recorded
    wind_speed_labels=['MS']
    vec_avg_wind_dir=['MD']
    wind_dir_labels=[] #this is NOT-vector average wind direction
    temp_unasp1_labels=['T']
    temp_unasp2_labels=[]
    temp_asp1_labels=['TA']
    temp_asp2_labels=[]
    precip_stage_labels=[]
    precip_weighing_labels=['PC']
    rel_hum_labels=[]
    barometer_labels=[] #no barometer at this station
    logger_bat_label=['VB']
    logger_temp_label=[]
    bad_labels=[]
    radiation_up_labels=['RU']
    radiation_down_labels=['RD']
    snow_depth_labels=['DS']
    
if (Glacier + Station=='Gulkana1920') | (Glacier + Station=='Wolverine1420'): #same program and same variable naming at JWS stations
    wind_gust_labels=[]#no wind gust recorded
    wind_speed_labels=['WS_ms_S_WVT']
    vec_avg_wind_dir=['WindDir_D1_WVT'] #I am guessing this is vec acerage; should come back and check program before using
    wind_dir_labels=[] #this is NOT-vector average wind direction
    temp_unasp1_labels=['AirTempC_Avg']
    temp_unasp2_labels=[]
    temp_asp1_labels=[]
    temp_asp2_labels=[]
    precip_stage_labels=[]
    precip_weighing_labels=[]
    rel_hum_labels=['RH']
    barometer_labels=['BP_inHg_Avg']
    logger_bat_label=['BattV_Avg']
    logger_temp_label=[]
    bad_labels=[]
    radiation_up_labels=['PyrnUp_Avg']
    radiation_down_labels=['PyrnDn_Avg']
    snow_depth_labels=[]

In [16]:
#Rename instruments 
dat.loc[dat.Instrument.isin(wind_gust_labels), "Instrument"]='WindGustSpeed'
dat.loc[dat.Instrument.isin(wind_speed_labels), "Instrument"]='WindSpeed'
dat.loc[dat.Instrument.isin(wind_dir_labels), "Instrument"]='WindDir'
dat.loc[dat.Instrument.isin(vec_avg_wind_dir), "Instrument"]='VecAvgWindDir'
dat.loc[dat.Instrument.isin(temp_unasp1_labels), "Instrument"]='Tpassive1'
dat.loc[dat.Instrument.isin(temp_unasp2_labels), "Instrument"]='Tpassive2'
dat.loc[dat.Instrument.isin(temp_asp1_labels), "Instrument"]='TAspirated1'
dat.loc[dat.Instrument.isin(temp_asp2_labels), "Instrument"]='TAspirated2'
dat.loc[dat.Instrument.isin(precip_stage_labels), "Instrument"]='StageCumulative'
dat.loc[dat.Instrument.isin(precip_weighing_labels), "Instrument"]='TPGCumulative'
dat.loc[dat.Instrument.isin(rel_hum_labels), "Instrument"]='RelHum'
dat.loc[dat.Instrument.isin(barometer_labels), "Instrument"]='Barom'
dat.loc[dat.Instrument.isin(logger_bat_label), "Instrument"]='LoggerBattery'
dat.loc[dat.Instrument.isin(logger_temp_label), "Instrument"]='LoggerTemp'
dat.loc[dat.Instrument.isin(radiation_up_labels), "Instrument"]='RadiationIn'
dat.loc[dat.Instrument.isin(radiation_down_labels), "Instrument"]='RadiationOut'
dat.loc[dat.Instrument.isin(snow_depth_labels), "Instrument"]='SnowDepth'

#Remove rows with bad data
dat=dat[~dat.Instrument.isin(bad_labels)] #Remove where labels are bad; introduced during maintenance

In [17]:
#Cast the data from long -> wide format
dat=dat.reset_index() #reset index so all unique
wide_dat=dat.pivot_table(columns='Instrument', values='Value', index='local_time')

In [18]:
wide_dat.head(1)

Instrument,LoggerBattery,RadiationIn,RadiationOut,SnowDepth,TAspirated1,TPGCumulative,Tpassive1,VecAvgWindDir,WindSpeed
local_time,,,,,,,,,
2013-03-19 16:45:00-08:00,14.2,NaN,NaN,NaN,-24.1,NaN,NaN,253.0,7.6


In [19]:
wide_dat.columns

Index(['LoggerBattery', 'RadiationIn', 'RadiationOut', 'SnowDepth',
       'TAspirated1', 'TPGCumulative', 'Tpassive1', 'VecAvgWindDir',
       'WindSpeed'],
      dtype='object', name='Instrument')

In [20]:
wide_dat.index.duplicated().any()

False

In [21]:
#List columns desired in output
# out_columns=['UTC_time', 'local_time', 'Tpassive1', 'Tpassive2',
#        'TAspirated1', 'TAspirated2', 'RelHum', 'StageCumulative',
#        'TPGCumulative', 'WindSpeed', 'WindGustSpeed', 'WindDir', 'Barom',
#         'VecAvgWindDir', 'RadiationIn', 'RadiationOut', 'SnowDepth', 'LoggerTemp', 'LoggerBattery']

In [22]:
#This should NO LONGER be neccessary (~4/18/18); future processing can eliminate the all NAN columns
#If this columns is not in dataframe, create
# for col in out_columns:
#     if col not in wide_dat.columns:
#         wide_dat[col]=np.nan #create the column; fill with NANs

In [23]:
#Reindex to 15min to ensure no timesteps are skipped
full_range_15_min = pd.date_range(wide_dat.index[0], wide_dat.index[-1], freq='15min')

In [24]:
wide_dat=wide_dat.reindex(index=full_range_15_min, fill_value=pd.np.nan)

In [25]:
#Set output format of time
wide_dat['UTC_time']=wide_dat.index.tz_convert('UTC').strftime(out_date_format)#Create column for true local time (as string, not UTC - X hrs)
wide_dat['local_time']=wide_dat.index.tz_convert(timezone).strftime(out_date_format)#Create column for true local time (as string, not UTC - X hrs)

In [26]:
wide_dat=wide_dat.reset_index(drop=True).copy() #drop index in output dataframe

In [27]:
save_dat=wide_dat

In [28]:
#directory to save output data
save_pth=base_path+ r"Data/" +Glacier+ r"/AllYears/Wx/LVL0/"
out_nm=Glacier.lower()+ Station + file_label+yr+".csv"

In [29]:
save_dat.to_csv(os.path.join(save_pth, out_nm), index=False, float_format='%g')
print("saved to: ", out_nm)

saved to:  gulkana1725_15min_all.csv


In [30]:
#If this is the Nunatak station, fill gaps with Irridium telemetry
if Glacier + Station=='Gulkana1725':
    %run "./Gulkana1725_FillGapsWithIrridium.py"

Gaps in data at Gulkana 1725 filled with Irridium telemetry data


In [31]:
#If Wolv 990 or Gulkana 1480, add 1990s data from NWIS to extend record back in time
if (Glacier + Station=='Wolverine990') | (Glacier + Station=='Gulkana1480'):    
    %run "./Add1990sDataFromNWIS.py"

In [32]:
save_dat.tail()

Instrument,LoggerBattery,RadiationIn,RadiationOut,SnowDepth,TAspirated1,TPGCumulative,Tpassive1,VecAvgWindDir,WindSpeed,UTC_time,local_time
178627,13.5,278.13,182.93,1.911,-3.4,0.7940,-2.4,258.0,0.7,2018/04/23 17:30,2018/04/23 09:30
178628,13.7,324.75,210.64,1.921,-3.1,0.7941,-1.2,172.0,0.3,2018/04/23 17:45,2018/04/23 09:45
178629,13.8,314.23,203.86,1.918,-4.2,0.7947,-2.3,193.0,0.6,2018/04/23 18:00,2018/04/23 10:00
178630,13.9,365.76,226.92,1.917,-4.5,0.7945,-2.5,200.0,2.1,2018/04/23 18:15,2018/04/23 10:15
178631,14.0,352.79,225.17,1.934,-3.9,0.8031,-1.1,182.0,0.7,2018/04/23 18:30,2018/04/23 10:30
